# Analytical Dataset (ADS)

An `ADS` is a table created for specific analytic purposes. The concept is to merge different data sources so that all possible information about the objects of interest (most often the clients) are in one place. This data is then distilled with sliding windows.

1. The core part of `ADS` is a sliding window for each time period (eg. 1 week).
2. `ADS` contains one line for each observation every week.

![graph](https://i.imgur.com/ojRMfB0.png)

The advantages of `ADS` is as follows:
- combines all data sources into one table
- in the future, ML models can be based on one table
- by using time slices (weekly, monthly), we take care of fluctuations in the data
- it suits as aggregation layer for the reporting
- batch scoring (weekly, monthly) is easy to implement
- new data sources can be simple added in the future using joins

![graph](https://i.imgur.com/zXsvJ25.png)

## Connecting to the `northwind` database

In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(path):
  con = None
  try:
    con = sqlite3.connect(database=path)
    print('Connection to SQLite DB successful.')
  except Error as e:
    print(f'The error \'{e}\' occurred.')
  
  return con

In [3]:
con = create_connection('./_data/northwind.db')

Connection to SQLite DB successful.


In [4]:
def execute_query(connection, query):
  cur = connection.cursor()
  result = None
  try:
    cur.execute(query)
    result = cur.fetchall()
    return result
    print('Query executed successfully.')
  except Error as e:
    print(f'The error \'{e}\' occurred.')

In [5]:
def execute_commit(connection, commit):
  cur = connection.cursor()
  try:
    cur.execute(commit)
    connection.commit()
    print('Query executed successfully.')
  except Error as e:
    print(f'The error \'{e}\' occurred.')

In [6]:
query_count = """ 
SELECT COUNT(*) FROM orders
"""

query_min = """ 
SELECT MIN(orderdate) FROM orders
"""

query_max = """ 
SELECT MAX(orderdate) FROM orders
"""

In [7]:
order_count = execute_query(con, query_count)
min_orderdate = execute_query(con, query_min)
max_orderdate = execute_query(con, query_max)
print(f'order count: {order_count[0][0]}\nmin order date: {min_orderdate[0][0]}\nmax order date: {max_orderdate[0][0]}')

order count: 830
min order date: 1996-07-04
max order date: 1998-05-06


There are 830 orders ranging from `1996-07-04` to `1998-05-06`. From this, an `ADS` can be built aggregated by month. \
It is also possible to aggregate by day or week but for this example, monthly windows are sufficient.

For traditional banking, 1 month may be enough. For telecommunications, 1 week can be appropriate, but there are also industries like e-commerce where they need to aggregate per day.

In this tutorial, orders will be aggregated each month and labeled with the column called `end_obs_date` (end observation date).

Example:
- order date: 1996-12-12 --> `endobsdate`: 1997-01-01
- order date: 1997-01-31 --> `endobsdate`: 1997-02-01

In [8]:
drop_endobsdate = """
DROP TABLE if exists end_obs_dates
"""

create_endobsdate = """ 
CREATE TABLE end_obs_dates
AS

WITH RECURSIVE
  cnt(x) AS (
    -- count begins at 0
    SELECT 0
    -- combine with value below, including duplicates
    UNION ALL
    -- count iterates by +1 for every recursion
    SELECT x+1 FROM cnt
    -- recursion ends when it meets the limit condition below,
    -- start and end date difference in days, then divided by 30 to return numbers of months and then +1
    LIMIT (SELECT ROUND(((julianday('1998-06-01') - julianday('1996-08-01'))/30) + 1))
    -- x is returned below for every recursion, adding a row with x value under the column 'end_obs_date' in table 'end_obs_dates'
    ) SELECT date('1996-08-01', '+' || x || ' month') AS end_obs_date FROM cnt
"""

In [9]:
execute_commit(con, drop_endobsdate) # drop table ensures that the table is not recreated below
execute_commit(con, create_endobsdate)

Query executed successfully.
Query executed successfully.


In [10]:
test_query_endobsdate = """ 
SELECT * FROM end_obs_dates
"""

dates = execute_query(con, test_query_endobsdate)
for date in dates:
  print(date)

('1996-08-01',)
('1996-09-01',)
('1996-10-01',)
('1996-11-01',)
('1996-12-01',)
('1997-01-01',)
('1997-02-01',)
('1997-03-01',)
('1997-04-01',)
('1997-05-01',)
('1997-06-01',)
('1997-07-01',)
('1997-08-01',)
('1997-09-01',)
('1997-10-01',)
('1997-11-01',)
('1997-12-01',)
('1998-01-01',)
('1998-02-01',)
('1998-03-01',)
('1998-04-01',)
('1998-05-01',)
('1998-06-01',)


In [11]:
drop_ads_population_hist = """ 
DROP TABLE if exists ads_population_hist
"""

create_ads_population_hist = """ 
CREATE TABLE ads_population_hist
AS
SELECT
  A.*,
  B.*
FROM end_obs_dates AS A
CROSS JOIN (
  SELECT DISTINCT customerid FROM customers) AS B
"""

execute_commit(con, drop_ads_population_hist)
execute_commit(con, create_ads_population_hist)

Query executed successfully.
Query executed successfully.


In [12]:
test_query_ads_population_hist = """ 
SELECT * FROM ads_population_hist
LIMIT 10
"""

hists = execute_query(con, test_query_ads_population_hist)
for hist in hists:
  print(hist)

('1996-08-01', 'ALFKI')
('1996-08-01', 'ANATR')
('1996-08-01', 'ANTON')
('1996-08-01', 'AROUT')
('1996-08-01', 'BERGS')
('1996-08-01', 'BLAUS')
('1996-08-01', 'BLONP')
('1996-08-01', 'BOLID')
('1996-08-01', 'BONAP')
('1996-08-01', 'BOTTM')


The primary goal is to create a table where all important information about clients is kept.
To do this, the following variables will be created:
- noofitems
- noofdistinct_orders
- total_price

All of which will be aggregated monthly.

First, compute the additional attribute `totalprice_for_product` as `unitprice * quantity`.

In [13]:
query_order_details_with_totalprice = """
SELECT *, unitprice*quantity AS totalprice_for_product
FROM 'Order Details'
LIMIT 10
"""

execute_query(con, query_order_details_with_totalprice)

[(10248, 11, 14.0, 12, 0.0, 168.0),
 (10248, 42, 9.8, 10, 0.0, 98.0),
 (10248, 72, 34.8, 5, 0.0, 174.0),
 (10249, 14, 18.6, 9, 0.0, 167.4),
 (10249, 51, 42.4, 40, 0.0, 1696.0),
 (10250, 41, 7.7, 10, 0.0, 77.0),
 (10250, 51, 42.4, 35, 0.15, 1484.0),
 (10250, 65, 16.8, 15, 0.15, 252.0),
 (10251, 22, 16.8, 6, 0.05, 100.80000000000001),
 (10251, 57, 15.6, 15, 0.05, 234.0)]

But the above does not summarize the performance of each order neatly. The orders are separated by each item within the order. \
Below, the products are grouped by `orderid` instead.

In [14]:
query_order_details_by_orderid = """
SELECT
  A.orderid,
  COUNT(DISTINCT A.productid) AS no_of_distinct_products,
  SUM(A.quantity) AS no_of_items,
  SUM(A.totalprice_for_product) AS total_price 
FROM (
  SELECT
  *,
  unitprice*quantity AS totalprice_for_product
  FROM "Order Details") AS A
GROUP BY 1
LIMIT 10
"""

execute_query(con, query_order_details_by_orderid)

[(10248, 3, 27, 440.0),
 (10249, 2, 49, 1863.4),
 (10250, 3, 60, 1813.0),
 (10251, 3, 41, 670.8),
 (10252, 3, 105, 3730.0),
 (10253, 3, 102, 1444.8000000000002),
 (10254, 3, 57, 625.2),
 (10255, 4, 110, 2490.5),
 (10256, 2, 27, 517.8),
 (10257, 3, 46, 1119.9)]

In [15]:
select_orders = """ 
SELECT
  orderid,
  customerid,
  orderdate
FROM orders
LIMIT 10
"""

execute_query(con, select_orders)

[(10248, 'VINET', '1996-07-04'),
 (10249, 'TOMSP', '1996-07-05'),
 (10250, 'HANAR', '1996-07-08'),
 (10251, 'VICTE', '1996-07-08'),
 (10252, 'SUPRD', '1996-07-09'),
 (10253, 'HANAR', '1996-07-10'),
 (10254, 'CHOPS', '1996-07-11'),
 (10255, 'RICSU', '1996-07-12'),
 (10256, 'WELLI', '1996-07-15'),
 (10257, 'HILAA', '1996-07-16')]

Assign `end_obs_date` to each order.

In [27]:
query_orders_with_endobsdate = """ 
SELECT
  orderid,
  customerid,
  orderdate,
  date(orderdate, 'start of month', '+1 month') AS end_obs_date,
  date(orderdate, 'start of month', '+3 months') AS end_obs_date_3M 
FROM orders
LIMIT 10
"""

execute_query(con, query_orders_with_endobsdate)

[(10248, 'VINET', '1996-07-04', '1996-08-01', '1996-10-01'),
 (10249, 'TOMSP', '1996-07-05', '1996-08-01', '1996-10-01'),
 (10250, 'HANAR', '1996-07-08', '1996-08-01', '1996-10-01'),
 (10251, 'VICTE', '1996-07-08', '1996-08-01', '1996-10-01'),
 (10252, 'SUPRD', '1996-07-09', '1996-08-01', '1996-10-01'),
 (10253, 'HANAR', '1996-07-10', '1996-08-01', '1996-10-01'),
 (10254, 'CHOPS', '1996-07-11', '1996-08-01', '1996-10-01'),
 (10255, 'RICSU', '1996-07-12', '1996-08-01', '1996-10-01'),
 (10256, 'WELLI', '1996-07-15', '1996-08-01', '1996-10-01'),
 (10257, 'HILAA', '1996-07-16', '1996-08-01', '1996-10-01')]

Combine into a table called `ads_orders_hist`.

In [17]:
drop_ads_orders_hist = """ 
DROP TABLE if exists ads_orders_hist
"""

create_ads_orders_hist = """
CREATE TABLE ads_orders_hist
AS
SELECT
  A.orderid,
  A.customerid,
  A.end_obs_date,
  A.end_obs_date_3M,
  B.no_of_distinct_products,
  B.no_of_items,
  B.total_price
FROM (
  SELECT
    orderid,
    customerid,
    orderdate,
    DATE(orderdate, 'start of month', '+1 month') AS end_obs_date,
    DATE(orderdate, 'start of month', '+3 months') AS end_obs_date_3m
  FROM orders) AS A
LEFT OUTER JOIN (
  SELECT
    A.orderid,
    COUNT(DISTINCT A.productid) AS no_of_distinct_products,
    SUM(A.quantity) AS no_of_items,
    SUM(A.totalprice_for_product) AS total_price
  FROM (
    SELECT
      *,
      unitprice*quantity AS totalprice_for_product
    FROM 'Order Details') AS A
  GROUP BY 1) AS B
ON A.orderid = B.orderid
"""

execute_commit(con, drop_ads_orders_hist)
execute_commit(con, create_ads_orders_hist)

Query executed successfully.
Query executed successfully.


Test that every `orderid` appears only one in the table.

In [18]:
select_ads_orders_hist = """ 
SELECT * FROM ads_orders_hist
"""

execute_query(con, select_ads_orders_hist)

[(10248, 'VINET', '1996-08-01', '1996-10-01', 3, 27, 440.0),
 (10249, 'TOMSP', '1996-08-01', '1996-10-01', 2, 49, 1863.4),
 (10250, 'HANAR', '1996-08-01', '1996-10-01', 3, 60, 1813.0),
 (10251, 'VICTE', '1996-08-01', '1996-10-01', 3, 41, 670.8),
 (10252, 'SUPRD', '1996-08-01', '1996-10-01', 3, 105, 3730.0),
 (10253, 'HANAR', '1996-08-01', '1996-10-01', 3, 102, 1444.8000000000002),
 (10254, 'CHOPS', '1996-08-01', '1996-10-01', 3, 57, 625.2),
 (10255, 'RICSU', '1996-08-01', '1996-10-01', 4, 110, 2490.5),
 (10256, 'WELLI', '1996-08-01', '1996-10-01', 2, 27, 517.8),
 (10257, 'HILAA', '1996-08-01', '1996-10-01', 3, 46, 1119.9),
 (10258, 'ERNSH', '1996-08-01', '1996-10-01', 3, 121, 2018.6),
 (10259, 'CENTC', '1996-08-01', '1996-10-01', 2, 11, 100.8),
 (10260, 'OTTIK', '1996-08-01', '1996-10-01', 4, 102, 1746.2),
 (10261, 'QUEDE', '1996-08-01', '1996-10-01', 2, 40, 448.0),
 (10262, 'RATTC', '1996-08-01', '1996-10-01', 3, 29, 624.8),
 (10263, 'ERNSH', '1996-08-01', '1996-10-01', 4, 184, 2464.8

In [19]:
test_ads_orders_hist = """ 
SELECT
  orderid,
  count(*)
FROM ads_orders_hist
-- group by the first column, 'orderid'
GROUP BY 1
-- order by the second column, 'orderid' count from highest to lowest
ORDER BY 2 DESC
LIMIT 5
"""

execute_query(con, test_ads_orders_hist)

[(11077, 1), (11076, 1), (11075, 1), (11074, 1), (11073, 1)]

The last step is to join `ads_orders_hist` with `ads_pop_hist`. 

In [20]:
drop_ads_observation_hist = """
DROP TABLE if exists ads_observation_hist
"""

create_ads_observation_hist = """
CREATE TABLE ads_observation_hist
AS
SELECT
  A.*,
  -- missing values represents no client orders for a given month and can be replaced with 0
  coalesce(B.no_of_distinct_orders_1M, 0) AS no_of_distinct_orders_1M,
  coalesce(B.no_of_items_1M, 0) AS no_of_items_1M,
  coalesce(B.avg_of_items_1M, 0) AS avg_of_items_1M,
  coalesce(B.total_price_1M, 0) AS total_price_1M,
  coalesce(B.max_total_price_1M, 0) AS max_total_price_1M,
  coalesce(B.min_total_price_1M, 0) AS min_total_price_1M
FROM ads_population_hist AS A
LEFT OUTER JOIN(
  -- group by orders, then customer
  SELECT
    customerid,
    end_obs_date,
    COUNT(DISTINCT orderid) AS no_of_distinct_orders_1M,
    SUM(no_of_items) AS no_of_items_1M,
    AVG(no_of_items) AS avg_of_items_1M,
    SUM(total_price) AS total_price_1M,
    MAX(total_price) AS max_total_price_1M,
    MIN(total_price) AS min_total_price_1M
  FROM ads_orders_hist
  GROUP BY 1, 2) AS B
ON
  A.customerid = B.customerid
AND
  A.end_obs_date = B.end_obs_date
"""

execute_commit(con, drop_ads_observation_hist)
execute_commit(con, create_ads_observation_hist)

Query executed successfully.
Query executed successfully.


Test if every `customerid` appears only once for every month.

In [21]:
test_ads_observation_hist = """
SELECT
  customerid,
  end_obs_date,
  count(*)
FROM ads_observation_hist
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 5
"""

execute_query(con, test_ads_observation_hist)

[('ALFKI', '1996-08-01', 1),
 ('ALFKI', '1996-09-01', 1),
 ('ALFKI', '1996-10-01', 1),
 ('ALFKI', '1996-11-01', 1),
 ('ALFKI', '1996-12-01', 1)]

In [22]:
select_ads_observation_hist = """ 
SELECT * FROM ads_observation_hist
WHERE no_of_distinct_orders_1M IS NOT 0
LIMIT 10
"""

execute_query(con, select_ads_observation_hist)

[('1996-08-01', 'BLONP', 1, 50, 50.0, 1176.0, 1176.0, 1176.0),
 ('1996-08-01', 'CENTC', 1, 11, 11.0, 100.8, 100.8, 100.8),
 ('1996-08-01', 'CHOPS', 1, 57, 57.0, 625.2, 625.2, 625.2),
 ('1996-08-01', 'ERNSH', 2, 305, 152.5, 4483.4, 2464.8, 2018.6),
 ('1996-08-01', 'FOLKO', 1, 60, 60.0, 724.5, 724.5, 724.5),
 ('1996-08-01', 'FRANK', 1, 135, 135.0, 4031.0, 4031.0, 4031.0),
 ('1996-08-01', 'GROSR', 1, 14, 14.0, 1101.2, 1101.2, 1101.2),
 ('1996-08-01', 'HANAR', 2, 162, 81.0, 3257.8, 1813.0, 1444.8000000000002),
 ('1996-08-01', 'HILAA', 1, 46, 46.0, 1119.9, 1119.9, 1119.9),
 ('1996-08-01', 'OTTIK', 1, 102, 102.0, 1746.2, 1746.2, 1746.2)]

In [23]:
drop_ads_observation_hist_3M = """
DROP TABLE if exists ads_observation_hist_3M
"""

create_ads_observation_hist_3M = """
CREATE TABLE ads_observation_hist_3M
AS
SELECT
  A.*,
  -- missing values represents no client orders for a given month and can be replaced with 0
  coalesce(B.no_of_distinct_orders_3M, 0) AS no_of_distinct_orders_3M,
  coalesce(B.no_of_items_3M, 0) AS no_of_items_3M,
  coalesce(B.avg_of_items_3M, 0) AS avg_of_items_3M,
  coalesce(B.total_price_3M, 0) AS total_price_3M,
  coalesce(B.max_total_price_3M, 0) AS max_total_price_3M,
  coalesce(B.min_total_price_3M, 0) AS min_total_price_3M
FROM ads_population_hist AS A
LEFT OUTER JOIN(
  -- group by orders, then customer
  SELECT
    customerid,
    end_obs_date_3M,
    COUNT(DISTINCT orderid) AS no_of_distinct_orders_3M,
    SUM(no_of_items) AS no_of_items_3M,
    AVG(no_of_items) AS avg_of_items_3M,
    SUM(total_price) AS total_price_3M,
    MAX(total_price) AS max_total_price_3M,
    MIN(total_price) AS min_total_price_3M
  FROM ads_orders_hist
  GROUP BY 1, 2) AS B
ON
  A.customerid = B.customerid
AND
  A.end_obs_date_3M = B.end_obs_date
"""

execute_commit(con, drop_ads_observation_hist_3M)
execute_commit(con, create_ads_observation_hist_3M)

Query executed successfully.
The error 'no such column: B.end_obs_date' occurred.


In [24]:
select_ads_observation_hist_3M = """ 
SELECT * FROM ads_observation_hist_3M
WHERE no_of_distinct_orders_3M IS NOT 0
LIMIT 10
"""

execute_query(con, select_ads_observation_hist_3M)

The error 'no such table: ads_observation_hist_3M' occurred.
